In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback , ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from tensorflow.keras.optimizers import (
    SGD,          # Stochastic Gradient Descent
    RMSprop,      # Root Mean Square Propagation
    Adam,         # Adaptive Moment Estimation
    AdamW,        # Adam with Weight Decay
    Nadam,        # Nesterov-accelerated Adam
    Adagrad,      # Adaptive Gradient Algorithm
    Adadelta,     # Adaptive Delta
    Ftrl          # Follow-the-Regularized-Leader
)

2025-08-05 10:16:57.525209: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-05 10:16:57.526236: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-05 10:16:57.529810: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-05 10:16:57.537115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754378217.548253   19138 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754378217.55

In [2]:
# Load data from the file
my_file = "./pentilfuran.MDE"
df = pd.read_csv(my_file, delim_whitespace=True, comment='#', names=["Step", "T", "E_KS", "E_tot", "Vol", "P"])

## Filter and concatenate data based on 'Step'
#df_data = pd.DataFrame()
#for i in range(1, 100):
#    df1 = df[df['Step'] == i].reset_index(drop=True)
#    df_data = pd.concat([df_data, df1[1:901]], ignore_index=True)
#
en = df['E_KS']
step =df.index[0:len(df['Step'])]


#? Convert the extracted data into a DataFrame for processing
data = pd.DataFrame({
    'Step': step.values,
    'Energy': en.values
})

#? Preprocessing or converting data in (0,1) interval
scaler = MinMaxScaler()
data['Energy_scaled'] = scaler.fit_transform(data[['Energy']])
print(len(data))

93300


/tmp/ipykernel_19138/3227072334.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(my_file, delim_whitespace=True, comment='#', names=["Step", "T", "E_KS", "E_tot", "Vol", "P"])


In [3]:
# Function to create time-series sequences
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:(i + 5000)]
        target = data[(i + 5000):(i + 10000)] #? the target is the second half of the data
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

#! Define sequence length (important parameter)
sequence_length = 10000
OUT_STEPS = 5000  # Number of future steps to predict
num_features = 1

sequences, targets = create_sequences(data['Energy_scaled'].values, sequence_length)

In [4]:
X_train, y_train = sequences[:67000], targets[:67000]
X_val, y_val = sequences[67000:87000], targets[67000:87000]
X_test, y_test = sequences[87000:], targets[87000:]

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

print(" Shape X train test val",X_train.shape,X_test.shape, X_val.shape)
print(" Shape Y train test val",y_train.shape,y_test.shape, y_val.shape)

 Shape X train test val (67000, 5000, 1) (0, 5000, 1) (16300, 5000, 1)
 Shape Y train test val (67000, 5000) (0, 5000) (16300, 5000)


In [5]:
# Define the Multi-Step CNN Model
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),  # Take the last CONV_WIDTH time steps
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=CONV_WIDTH),  # Conv1D layer
    tf.keras.layers.Dense(OUT_STEPS * num_features, kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

# Callbacks for training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath="best_autoencoder_model.keras",
    monitor='val_loss',
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

# Compile and Fit function
def compile_and_fit(model, X_train, y_train, X_val, y_val, epochs=20, batch_size=10):
    model.compile(optimizer=Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-7), loss='mse')
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val),callbacks=[early_stopping, model_checkpoint])
    return history

# Train the model
history = compile_and_fit(multi_conv_model, X_train, y_train, X_val, y_val, epochs=20, batch_size=10)

test_loss = multi_conv_model.evaluate(X_val, y_val)
print(f'Test Loss: {test_loss}')




W0000 00:00:1754378258.429969   19138 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


: 

In [13]:
predictions = multi_conv_model.predict(X_test)
print(predictions.shape,X_test.shape,y_test.shape)
# Invert scaling for predictions and actual values
y_test_inverted = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inverted = scaler.inverse_transform(predictions.reshape(-1, 1))
print(predictions_inverted.shape,y_test_inverted.shape)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=y_test[0:,3], mode='lines', name='True Values'))
fig1.add_trace(go.Scatter(y=predictions[0:,3,0], mode='lines', name='Predicted Values'))
fig1.update_layout(title='True vs Predicted Energy (CNN)', xaxis_title='Index', yaxis_title='Energy')
fig1.write_html(file="test_cnn.html", auto_open=False)
fig1.show()

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(2000, 50, 1) (2000, 50, 1) (2000, 50)
(100000, 1) (100000, 1)


In [11]:
predictions = multi_conv_model.predict(X_test)
print(predictions.shape,X_test.shape,y_test.shape)
# Invert scaling for predictions and actual values
y_test_inverted = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inverted = scaler.inverse_transform(predictions.reshape(-1, 1))
print(predictions_inverted.shape,y_test_inverted.shape)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=y_test[0:,3], mode='lines', name='True Values'))
fig1.add_trace(go.Scatter(y=predictions[0:,3,0], mode='lines', name='Predicted Values'))
fig1.update_layout(title='True vs Predicted Energy (CNN)', xaxis_title='Index', yaxis_title='Energy')
fig1.write_html(file="test_cnn.html", auto_open=False)
fig1.show()

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step
(2000, 50, 1) (2000, 50, 1) (2000, 50)
(100000, 1) (100000, 1)


In [8]:
predictions = multi_conv_model.predict(X_test)
print(predictions.shape,X_test.shape,y_test.shape)
# Invert scaling for predictions and actual values
y_test_inverted = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inverted = scaler.inverse_transform(predictions.reshape(-1, 1))
print(predictions_inverted.shape,y_test_inverted.shape)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=y_test[0:,3], mode='lines', name='True Values'))
fig1.add_trace(go.Scatter(y=predictions[0:,3,0], mode='lines', name='Predicted Values'))
fig1.update_layout(title='True vs Predicted Energy (CNN)', xaxis_title='Index', yaxis_title='Energy')
fig1.write_html(file="test_cnn.html", auto_open=False)
fig1.show()

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(2000, 50, 1) (2000, 50, 1) (2000, 50)
(100000, 1) (100000, 1)


In [6]:
predictions = multi_conv_model.predict(X_test)
print(predictions.shape,X_test.shape,y_test.shape)
# Invert scaling for predictions and actual values
y_test_inverted = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inverted = scaler.inverse_transform(predictions.reshape(-1, 1))
print(predictions_inverted.shape,y_test_inverted.shape)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=y_test[0:,3], mode='lines', name='True Values'))
fig1.add_trace(go.Scatter(y=predictions[0:,3,0], mode='lines', name='Predicted Values'))
fig1.update_layout(title='True vs Predicted Energy (CNN)', xaxis_title='Index', yaxis_title='Energy')
fig1.write_html(file="test_cnn.html", auto_open=False)
fig1.show()

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(2000, 50, 1) (2000, 50, 1) (2000, 50)
(100000, 1) (100000, 1)
